Author: Zhewei Yao <https://github.com/yaozhewei>, Amir Gholami <http://amirgholami.org/>


This tutorial shows how to compute the Hessian information using (randomized) numerical linear algebra for both explicit Hessian (the matrix is given) as well as implicit Hessian (the matrix is ungiven).

We'll start by doing the necessary imports:

In [4]:
import numpy as np
import torch 
import copy
from torchvision import datasets, transforms
from utils import * # get the dataset
from pyhessian import hessian # Hessian computation
from density_plot import get_esd_plot # ESD plot
from pytorchcv.model_provider import get_model as ptcv_get_model # model

import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# enable cuda devices
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# The following part shows how to use power iteration to get the top eigenvalue of a matrix without explicitly having access to it

## We first show an example with Numpy for a random matrix B

In [6]:
n = 10 # the matrix size

# generate the matrix
A = np.random.randn(n, n)
B = A @ A.T

Now let's use numpy to compute the ground truth eigenvalues. We will then check the results with this.

In [12]:
# use np.eigs to get the top eigenvalue of B
eigs, _ = np.linalg.eig(B)

print("The top eigenvalue of B is %.4f"%np.sort(eigs)[-1])

The top eigenvalue of B is 23.9784


Now let's try to comptue the top eigenvalue of B without explicitly using. To do so, we will use a method called Power Iteration:

https://en.wikipedia.org/wiki/Power_iteration

The algorithm is very simple and efficiet to compute the top eigenvalue:
$$v_{i+1} = \frac{Bv_i}{\|Bv_i\|}.$$

As such, we only need to have access to the *application* of B to a given vector $v_i$ and not the matrix B itself. This application is commonly referred to as *matvec* in literature.

In [21]:
# use power iteration to get the top eigenvalue of B
v = np.random.randn(n, 1)

for i in range(40):
    v = v / np.linalg.norm(v)
    print("Step %2d current estimated top eigvalue: %.4f"%(i+1,v.T @ B @ v))
    v = B @ v


Step  1 current estimated top eigvalue: 16.6602
Step  2 current estimated top eigvalue: 22.1014
Step  3 current estimated top eigvalue: 23.3392
Step  4 current estimated top eigvalue: 23.7363
Step  5 current estimated top eigvalue: 23.8803
Step  6 current estimated top eigvalue: 23.9371
Step  7 current estimated top eigvalue: 23.9607
Step  8 current estimated top eigvalue: 23.9707
Step  9 current estimated top eigvalue: 23.9751
Step 10 current estimated top eigvalue: 23.9769
Step 11 current estimated top eigvalue: 23.9778
Step 12 current estimated top eigvalue: 23.9781
Step 13 current estimated top eigvalue: 23.9783
Step 14 current estimated top eigvalue: 23.9784
Step 15 current estimated top eigvalue: 23.9784
Step 16 current estimated top eigvalue: 23.9784
Step 17 current estimated top eigvalue: 23.9784
Step 18 current estimated top eigvalue: 23.9784
Step 19 current estimated top eigvalue: 23.9784
Step 20 current estimated top eigvalue: 23.9784
Step 21 current estimated top eigvalue: 

As you can see the result of the power iteration and the one we got from numpy match very well.

We can apply the same techinique for neural networks as well, and in particular to Hessian!

Importantly there has been a lot of misconception that we can not use Hessian for real world applications since
we need to form it. Next you will see that this is not correct.

## Use power iteration for NNs

In [ ]:
# get the model 
model = ptcv_get_model("resnet20_cifar10", pretrained=True)
# change the model to eval mode to disable running stats upate
model.eval()

# create loss function
criterion = torch.nn.CrossEntropyLoss()

# get dataset 
train_loader, test_loader = getData()

# for illustrate, we only use one batch to do the tutorial
for inputs, targets in train_loader:
    break

# we use cuda to make the computation fast
model = model.cuda()
inputs, targets = inputs.cuda(), targets.cuda()



In [ ]:
# create the hessian computation module
hessian_comp = hessian(model, criterion, data=(inputs, targets), cuda=True)

In [ ]:
top_eigenvalues, top_eigenvector = hessian_comp.eigenvalues()
print(f'The top eigenvalue of this model is: {top_eigenvalues}')

In [ ]:
top_eigenvalues, top_eigenvector = hessian_comp.eigenvalues(top_n=2)
print(f'The top two eigenvalues of this model are: {top_eigenvalues}')

We can use the Hessian eigenvectors/eigenvalues to analyze the flat/sharpness of the loss landscape of your model, and plot the loss landscape.


In [ ]:
# get the top eigenvector
top_eigenvalues, top_eigenvector = hessian_comp.eigenvalues()

In [ ]:
# This is a simple function, that will allow us to perturb the model paramters and get the result
def get_params(model_orig,  model_perb, direction, alpha):
    for m_orig, m_perb, d in zip(model_orig.parameters(), model_perb.parameters(), direction):
        m_perb.data = m_orig.data + alpha * d
    return model_perb

In [ ]:
# lambda is a small scalar that we use to perturb the model parameters along the eigenvectors 
lams = np.linspace(-0.5, 0.5, 21).astype(np.float32)

loss_list = []

# create a copy of the model
model_perb = ptcv_get_model("resnet20_cifar10", pretrained=True)
model_perb.eval()
model_perb = model_perb.cuda()

for lam in lams:
    model_perb = get_params(model, model_perb, top_eigenvector[0], lam)
    loss_list.append(criterion(model_perb(inputs), targets).item())

plt.plot(lams, loss_list)
plt.ylabel('Loss')
plt.xlabel('Perturbation')

In [ ]:
from pyhessian.utils import normalization

# generate random vector to do the loss plot

v = [torch.randn_like(p) for p in model.parameters()]
v = normalization(v)


# used to perturb your model 
lams = np.linspace(-0.5, 0.5, 21).astype(np.float32)

loss_list = []

# create a copy of the model
model_perb = ptcv_get_model("resnet20_cifar10", pretrained=True)
model_perb.eval()
model_perb = model_perb.cuda()

for lam in lams: 
    model_perb = get_params(model, model_perb, v, lam)
    loss_list.append(criterion(model_perb(inputs), targets).item())

plt.plot(lams, loss_list)
plt.ylabel('Loss')
plt.xlabel('Perturbation')

In [ ]:
from pyhessian.utils import normalization




# used to perturb your model 
lams = np.linspace(-0.5, 0.5, 21).astype(np.float32)

loss_list = []

# create a copy of the model
model_perb = ptcv_get_model("resnet20_cifar10", pretrained=True)
model_perb.eval()
model_perb = model_perb.cuda()

# generate gradient vector to do the loss plot
loss = criterion(model_perb(inputs), targets)
loss.backward()

v = [p.grad.data for p in model_perb.parameters()]
v = normalization(v)
model_perb.zero_grad()


for lam in lams: 
    model_perb = get_params(model, model_perb, v, lam)
    loss_list.append(criterion(model_perb(inputs), targets).item())

plt.plot(lams, loss_list)
plt.ylabel('Loss')
plt.xlabel('Perturbation')

# The following part shows how to use power iteration to get the trace/diagonal of the Hessian

## We first show an example with Numpy

In [ ]:
n = 1000 # the matrix size

# generate the matrix
A = np.random.randn(n, n)
B = A @ A.T

In [ ]:
# Direct get the trace 
print(f'The trace of B is: {np.matrix.trace(B)}')

## Hutchinson Method
The algorithm is also very simple as power iteration: 
$$Tr(B) = \mathbb{E}[v^TBv],$$
$$Diag(B) = \mathbb{E}[v \bigodot Bv].$$

In [ ]:
# use Hutchinson method to get the trace of B
trace_list = []

for i in range(20):
    v = np.random.randint(2, size=n)
    v = v.reshape(n, 1) * 2 - 1
    trace_list.append(v.T @ B @ v)
    print(f'Step {i+1}, Current estimated trace: {np.mean(trace_list)}, \
          the current relative error: { (np.mean(trace_list) - np.matrix.trace(B)) / np.matrix.trace(B)} ')


In [ ]:
# use Hutchinson method to get the diag of B
diag_est = np.zeros([n, 1])

for i in range(20):
    v = np.random.randint(2, size=n)
    v = v.reshape(n, 1) * 2 - 1
    diag_est += np.multiply(v, (B @ v))
    print(f'Step {i+1}, the current average relative error: { np.mean(np.abs(diag_est.reshape(-1) / (i+1) - np.diag(B)) / np.diag(B))} ')


## Use Hutchison Method for NNs

In [ ]:
# get the model 
model = ptcv_get_model("resnet20_cifar10", pretrained=True)
# change the model to eval mode to disable running stats upate
model.eval()

# create loss function
criterion = torch.nn.CrossEntropyLoss()

# get dataset 
train_loader, test_loader = getData()

# for illustrate, we only use one batch to do the tutorial
for inputs, targets in train_loader:
    break

# we use cuda to make the computation fast
model = model.cuda()
inputs, targets = inputs.cuda(), targets.cuda()



In [ ]:
# create the hessian computation module
hessian_comp = hessian(model, criterion, data=(inputs, targets), cuda=True)

In [ ]:
trace = hessian_comp.trace()
print(f'The trace of this model is: {np.mean(trace)}')

# The following section shows a more advantage example using stochastic lanczos algorithm.

In [ ]:
density_eigen, density_weight = hessian_comp.density()

In [ ]:
get_esd_plot(density_eigen, density_weight)